In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("./sample_data/searchkeyword.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188359 entries, 0 to 188358
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   searchkeyword  188359 non-null  object
 1   count          188359 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ MB


In [10]:
# df["searchkeyword"] = df["searchkeyword"].str.replace("\t", "").str.replace("\n", "").str.replace(" ", "").str.replace("-", "").str.replace("‘", "").str.strip()
# df

In [11]:
import re


def clean_text(text:str) -> str:
    return re.sub(r'[^\u4e00-\u9fa5]', '', text)

In [12]:
df["searchkeyword"] = df["searchkeyword"].apply(clean_text)
df

,searchkeyword,count
0,白泥碳酸潔面泡,1
1,潔廁劑原味,1
2,劇,1
3,喇叭牌正露丸粒,1
4,歐萊雅,1
...,...,...
188354,芝孢子,8
188355,芝抱子,4
188356,芝苞子,3
188357,燒鵝,1


In [13]:
filtered_df = df[(df["searchkeyword"].str.len() > 2) & (df["searchkeyword"].str.len() < 6)]
filtered_df

,searchkeyword,count
1,潔廁劑原味,1
4,歐萊雅,1
5,蘭州歸脾丸,1
9,益生菌,4
14,日本命力,1
...,...,...
188343,生骨骼,1
188349,適達牙膏,1
188354,芝孢子,8
188355,芝抱子,4


In [14]:
filtered_df = filtered_df.groupby("searchkeyword", as_index=False).agg({"count": "sum"}).sort_values(by="count", ascending=False).reset_index(drop=True)

In [15]:
filtered_df.to_csv("./n_gram_processed/filter_df.csv", index=False)

In [8]:
all_keywords = filtered_df["searchkeyword"].tolist()
all_count = filtered_df["count"].tolist()

In [9]:
# filtered_df['compare'] = filtered_df['searchkeyword'].agg(lambda x: all_keywords)
# filtered_df

In [10]:
def generate_regex_patterns(keyword):
    patterns = []
    for i in range(len(keyword)):
        # 在每個位置替換一個字符為'.'
        pattern = keyword[:i] + '.' + keyword[i+1:]
        patterns.append(pattern)
    return patterns

In [11]:
filtered_df["regex_pattern"] = filtered_df["searchkeyword"].apply(generate_regex_patterns)
filtered_df

,searchkeyword,count,regex_pattern
0,濕紙巾,22859,"[.紙巾, 濕.巾, 濕紙.]"
1,益生菌,18134,"[.生菌, 益.菌, 益生.]"
2,衛生巾,14198,"[.生巾, 衛.巾, 衛生.]"
3,維他命,14080,"[.他命, 維.命, 維他.]"
4,洗頭水,11657,"[.頭水, 洗.水, 洗頭.]"
...,...,...,...
68660,龜靈舒,1,"[.靈舒, 龜.舒, 龜靈.]"
68661,一分鐘髮,1,"[.分鐘髮, 一.鐘髮, 一分.髮, 一分鐘.]"
68662,一分鐘速效,1,"[.分鐘速效, 一.鐘速效, 一分.速效, 一分鐘.效, 一分鐘速.]"
68663,一分鐘白滑,1,"[.分鐘白滑, 一.鐘白滑, 一分.白滑, 一分鐘.滑, 一分鐘白.]"


In [12]:
# def compare_to_searchkeywords(patterns, keywords_list):  #　100 -> 8.7s
#     def find_matches(kw_pattern, kw_list):
#         pattern = re.compile(kw_pattern)
#         match_list = list(filter(lambda x: pattern.fullmatch(x) if x != kw_pattern else None, kw_list))
#         return match_list

#     temp_df = pd.DataFrame.from_dict(
#         {
#             "regex_pattern": patterns, 
#             "keywords_list": [keywords_list] * len(patterns)
#         }
#     )
#     temp_df["matches"] = temp_df.apply(lambda x: find_matches(x["regex_pattern"], x["keywords_list"]), axis=1)
#     return list(set(temp_df["matches"].sum()))

In [13]:
# def compare_to_searchkeywords(patterns, keywords_list):  #　100 -> 13s
#     def find_matches(kw_pattern, kw_list):
#         kw_series = pd.Series(kw_list)
#         pattern = re.compile(kw_pattern)
#         match_list = kw_series.apply(lambda x: x if pattern.search(x) else None).dropna().tolist()
#         return match_list

#     temp_df = pd.DataFrame.from_dict(
#         {
#             "regex_pattern": patterns, 
#             "keywords_list": [keywords_list] * len(patterns)
#         }
#     )
#     temp_df["matches"] = temp_df.apply(lambda x: find_matches(x["regex_pattern"], x["keywords_list"]), axis=1)
#     return list(set(temp_df["matches"].sum()))

In [14]:
def compare_to_searchkeywords(patterns, keywords_list):  #　100 -> 5.9s
    results = []
    for pattern in patterns:
        regex = re.compile(pattern)
        # 搜尋list中匹配的項目
        for word in keywords_list:
            # if len(word) == len(keyword) and word != keyword:
            if regex.fullmatch(word):
                results.append(word)
    if len(results) > 0:
        return results
    return None

In [15]:
filtered_df["matches"] = filtered_df["regex_pattern"].apply(lambda x: compare_to_searchkeywords(x, all_keywords))
filtered_df.to_csv("./n_gram_processed/compare_to_searchkeywords.csv", index=False)

In [16]:
filtered_df

,searchkeyword,count,regex_pattern,matches
0,濕紙巾,22859,"[.紙巾, 濕.巾, 濕紙.]","[濕紙巾, 嬰兒濕紙巾, 消毒濕紙巾, 酒精濕紙巾, 純水濕紙巾, 屈臣氏紙巾, 滴露濕紙巾..."
1,益生菌,18134,"[.生菌, 益.菌, 益生.]","[益生菌, 兒童益生菌, 女士益生菌, 孕婦益生菌, 嬰兒益生菌, 古寶益生菌, 善存益生菌..."
2,衛生巾,14198,"[.生巾, 衛.巾, 衛生.]","[衛生巾, 液體衛生巾, 衞生巾, 卫生巾, 產婦衛生巾, 液態衛生巾, 液體衞生巾, 夜用..."
3,維他命,14080,"[.他命, 維.命, 維他.]","[維他命, 维他命, 維他命雜, 兒童維他命, 綜合維他命, 維他命精華, 多種維他命, 維..."
4,洗頭水,11657,"[.頭水, 洗.水, 洗頭.]","[洗頭水, 惠洗頭水, 呂洗頭水, 施巴洗頭水, 去黃洗頭水, 沙宣洗頭水, 鎖色洗頭水, ..."
...,...,...,...,...
68660,龜靈舒,1,"[.靈舒, 龜.舒, 龜靈.]","[龜靈舒, 龜靈舒, 龜靈清, 五龜靈芝, 龜靈袪濕丸, 龜靈舒]"
68661,一分鐘髮,1,"[.分鐘髮, 一.鐘髮, 一分.髮, 一分鐘.]","[一分鐘髮膜, 一分鐘髮, 一分鐘髮膜, 一分鐘髮, 一分鐘髮膜, 一分鐘髮, 一分鐘焗油,..."
68662,一分鐘速效,1,"[.分鐘速效, 一.鐘速效, 一分.速效, 一分鐘.效, 一分鐘速.]","[一分鐘速效, 一分鐘速效, 一分鐘速效, 一分鐘速效, 一分鐘速效]"
68663,一分鐘白滑,1,"[.分鐘白滑, 一.鐘白滑, 一分.白滑, 一分鐘.滑, 一分鐘白.]","[一分鐘白滑, 一分鐘白滑, 一分鐘白滑, 一分鐘白滑, 一分鐘白滑]"


In [17]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68665 entries, 0 to 68664
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   searchkeyword  68665 non-null  object
 1   count          68665 non-null  int64 
 2   regex_pattern  68665 non-null  object
 3   matches        68665 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.1+ MB


In [25]:
expand_df = filtered_df[["searchkeyword", "count", "matches"]].explode("matches")
expand_df = expand_df[(expand_df["matches"].str.len() == expand_df["searchkeyword"].str.len()) & (expand_df["matches"] != expand_df["searchkeyword"])]
expand_df

,searchkeyword,count,matches
0,濕紙巾,22859,竹紙巾
0,濕紙巾,22859,乾紙巾
0,濕紙巾,22859,面紙巾
0,濕紙巾,22859,手紙巾
0,濕紙巾,22859,座紙巾
...,...,...,...
68658,龜頭災,1,龜頭炎
68659,龜靈袪濕丸,1,龜苓袪濕丸
68660,龜靈舒,1,龜靈清
68664,一分鈡焗油,1,一分鐘焗油


In [26]:
filtered_count_df = filtered_df[["searchkeyword", "count"]]
filtered_count_df.columns = ["matches", "count"]

expand_df = expand_df.merge(filtered_count_df, how="inner", on="matches")
expand_df

,searchkeyword,count_x,matches,count_y
0,濕紙巾,22859,竹紙巾,149
1,濕紙巾,22859,乾紙巾,17
2,濕紙巾,22859,面紙巾,16
3,濕紙巾,22859,手紙巾,11
4,濕紙巾,22859,座紙巾,9
...,...,...,...,...
273501,龜頭災,1,龜頭炎,7
273502,龜靈袪濕丸,1,龜苓袪濕丸,6
273503,龜靈舒,1,龜靈清,4
273504,一分鈡焗油,1,一分鐘焗油,25


In [40]:
# expand_df[((expand_df["count_y"] / expand_df["count_x"]) < 0.35) & (expand_df["count_x"] > 70)]
expand_df["processed"] = expand_df.apply(lambda x: x["searchkeyword"] if (x["count_y"]/x["count_x"]<0.35) and (x["count_x"]>70) else x["matches"], axis=1)
expand_df

,searchkeyword,count_x,matches,count_y,processed
0,濕紙巾,22859,竹紙巾,149,濕紙巾
1,濕紙巾,22859,乾紙巾,17,濕紙巾
2,濕紙巾,22859,面紙巾,16,濕紙巾
3,濕紙巾,22859,手紙巾,11,濕紙巾
4,濕紙巾,22859,座紙巾,9,濕紙巾
...,...,...,...,...,...
273501,龜頭災,1,龜頭炎,7,龜頭炎
273502,龜靈袪濕丸,1,龜苓袪濕丸,6,龜苓袪濕丸
273503,龜靈舒,1,龜靈清,4,龜靈清
273504,一分鈡焗油,1,一分鐘焗油,25,一分鐘焗油


In [42]:
expand_df[["matches", "processed"]].to_csv("./n_gram_processed/processed_searchkeywords.csv", index=False, encoding="utf-8-sig")